In [9]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.metrics import  accuracy_score
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, accuracy_score, auc
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_curve

from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold

In [5]:
path1=Path("/Users/zainabnazari/Desktop/pdbp/PPMI_PDBP_Log2CPM_sex_effect_removed_Norm_Quantile_together_PD_CTR")

In [30]:
pdbp_ppmi_data=pd.read_csv(path1/"PDBP_subset_from_PPMI_PDBP_shared_genes_Log2CPM_sex_effect_removed_QuantileNorm_02giu2024.txt",delimiter='\t')
t_pdbp = pdbp_ppmi_data.T

In [31]:
index_pdbp = t_pdbp.reset_index()

In [32]:
index_pdbp['index']= index_pdbp['index'].str[3:]

In [33]:
index_pdbp

,index,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000285687,ENSG00000285765,ENSG00000285814,ENSG00000285843,ENSG00000285891,ENSG00000285913,ENSG00000285937,ENSG00000285946,ENSG00000285950,ENSG00000285975
0,PDLK184WGE,-0.795371,-2.729347,4.132727,5.947212,3.915167,9.081336,2.544537,4.212770,5.115425,...,-2.784321,-2.838762,-2.786100,-2.942664,-2.770755,-2.770755,-2.788866,-2.887931,-2.825500,-2.781080
1,PDXW222GG2,0.029172,-3.487062,5.383310,6.113187,4.208315,8.994625,4.462823,4.380455,5.551275,...,-3.433885,-3.367082,-3.431357,-3.268287,-3.452658,-3.452658,-3.427209,-3.317961,-3.380948,-3.442536
2,PDDY096RP8,0.161952,-2.245680,4.764646,6.189538,4.310707,8.915631,3.514179,4.579706,5.807383,...,-3.481217,-3.462194,-3.480309,-3.362947,-3.490272,-3.490272,-3.479424,-3.199985,-3.393907,-3.483213
3,PDCJ149XWQ,-0.851656,-2.650620,5.398471,6.203822,4.209682,8.833031,3.559513,4.704305,5.508427,...,-2.240673,-2.789659,-2.730225,-2.912417,-2.710112,-2.710112,-2.733033,-2.850490,-2.775262,-2.722415
4,PDYH176ZFA,-0.712838,-2.724630,4.980514,6.184612,4.256929,8.860026,2.689762,4.447423,5.452709,...,-2.781719,-2.836982,-2.783305,-2.940982,-2.767595,-2.767595,-2.785704,-2.886320,-2.823342,-2.777919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,PDZA968YLU,-0.432641,-2.489323,4.779148,5.963757,4.270477,9.142878,2.490018,4.810756,5.216479,...,-2.589854,-2.695303,-2.593489,-2.857318,-2.565912,-2.565912,-2.597467,-2.775536,-2.670709,-2.583686
1203,PDBE349TEN,-0.769956,-2.580791,4.700159,5.839763,3.898746,9.267990,4.262164,4.266996,5.253896,...,-2.670956,-2.754169,-2.674132,-2.891556,-2.649150,-2.649150,-2.678160,-2.821396,-2.735409,-2.664074
1204,PDCJ314NKT,-0.075972,-3.483480,4.797046,6.283134,4.454279,8.577536,3.152542,4.022217,5.745774,...,-3.419326,-3.350476,-3.416596,-3.247988,-3.442413,-3.442413,-3.414100,-3.301836,-3.365560,-3.424378
1205,PDYZ819YLQ,-1.151518,-2.418979,5.472407,6.263525,4.380882,8.987690,2.924825,4.543833,5.547137,...,-2.521390,-2.639686,-2.525174,-2.833294,-2.494825,-2.494825,-2.530494,-2.740900,-2.612083,-2.513835


In [50]:
pdbp_ppmi_data1=pd.read_csv(path1/"PPMI_subset_from_PPMI_PDBP_shared_genes_Log2CPM_sex_effect_removed_QuantileNorm_02giu2024.txt",delimiter='\t')
t_ppmi = pdbp_ppmi_data1.T

In [51]:
t_ppmi.reset_index(inplace=True)

In [57]:
t_ppmi['index']= t_ppmi['index'].str[1:]

In [62]:
t_ppmi.rename(columns={'index':'ID'}, inplace=True)

In [64]:
t_ppmi.set_index('ID', inplace=True)

In [65]:
t_ppmi

,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000285687,ENSG00000285765,ENSG00000285814,ENSG00000285843,ENSG00000285891,ENSG00000285913,ENSG00000285937,ENSG00000285946,ENSG00000285950,ENSG00000285975
ID,,,,,,,,,,,,,,,,,,,,,
3000,-0.101375,-2.999518,4.328498,6.049779,4.211890,8.784764,2.173793,4.125101,5.434887,6.313419,...,-3.479269,-3.442632,-3.478343,-3.364637,-3.487534,-3.487534,-3.477212,-3.160921,-3.276003,-3.480959
3001,-0.070030,-2.736090,5.246508,6.163900,4.088958,8.628968,3.412540,4.231814,5.556669,6.403626,...,-2.790381,-2.845587,-2.791950,-2.947585,-2.776044,-2.776044,-2.794114,-2.892143,-2.832832,-2.785990
3002,1.259571,-1.404902,5.358119,6.074396,4.424164,8.762913,2.727398,4.769609,5.340044,6.252896,...,-3.481981,-3.455701,-3.481048,-3.323370,-3.490907,-3.490907,-3.479984,-2.779435,-2.944676,-3.483760
3003,-1.071352,-2.890159,5.247370,6.124415,4.354913,8.363715,4.113420,4.505387,6.446975,6.365839,...,-3.466844,-2.340830,-3.464768,-3.303630,-3.477475,-3.477475,-3.462194,-3.064134,-3.415254,-3.470379
3004,-0.499520,-2.414768,5.351478,6.352868,4.315031,9.066409,4.021306,4.289712,5.111581,6.509765,...,-3.332901,-3.356088,-3.333370,-3.078897,-3.322814,-3.322814,-3.333964,-2.935973,-2.932657,-3.329702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4102,0.394976,-3.020122,5.379363,6.271351,4.256049,8.136359,4.036716,4.581787,5.583952,6.584387,...,-3.048055,-2.786471,-3.049144,-2.816837,-3.039936,-3.039936,-3.050273,-3.101747,-1.829699,-3.045297
4108,-1.064643,-2.870947,4.891916,6.071242,4.519018,8.626706,3.253652,4.551400,6.067533,6.208213,...,-3.110857,-3.133537,-3.111776,-3.190143,-3.105350,-3.105350,-3.113076,-3.159333,-3.128504,-3.108994
4115,-0.626887,-2.947472,5.040913,6.138357,4.725125,8.664279,2.839370,4.178273,5.699001,6.455780,...,-2.977598,-3.008691,-2.978779,-3.080310,-2.969678,-2.969678,-2.979836,-2.605039,-3.001129,-2.975235


In [74]:
pdbp_ppmi_data2=pd.read_csv(path1/"factor_PPMI_subset_from_PPMI_PDBP_shared_genes_Log2CPM_sex_effect_removed_QuantileNorm_02giu2024.txt",delimiter='\t')
pdbp_ppmi_data2['ID'] = pdbp_ppmi_data2['ID'].astype(str)

0      3000
1      3001
2      3002
3      3003
4      3004
       ... 
540    4102
541    4108
542    4115
543    4136
544    4139
Name: ID, Length: 545, dtype: object

In [71]:
combined_ppmi = pd.merge(t_ppmi,pdbp_ppmi_data2, on='ID')

In [76]:
combined_ppmi.to_csv(path1/'ppmi_data.csv', sep='\t')

In [10]:
pdbp_ppmi_data3=pd.read_csv(path1/"factor_PDBP_subset_from_PPMI_PDBP_shared_genes_Log2CPM_sex_effect_removed_QuantileNorm_02giu2024.txt",delimiter='\t')
pdbp_ppmi_data3

,ID,Diagnosis,Sex
0,PD_PDLK184WGE,PD,M
1,PD_PDXW222GG2,PD,F
2,PD_PDDY096RP8,CTR,F
3,PD_PDCJ149XWQ,PD,M
4,PD_PDYH176ZFA,CTR,M
...,...,...,...
1202,PD_PDZA968YLU,PD,M
1203,PD_PDBE349TEN,CTR,M
1204,PD_PDCJ314NKT,CTR,F
1205,PD_PDYZ819YLQ,PD,M


### Joining the columns of diagnosis and Sex with ID

In [11]:
pdbp_ppmi_data3['ID'] = pdbp_ppmi_data3['ID'].str[3:]
pdbp_ppmi_data3

,ID,Diagnosis,Sex
0,PDLK184WGE,PD,M
1,PDXW222GG2,PD,F
2,PDDY096RP8,CTR,F
3,PDCJ149XWQ,PD,M
4,PDYH176ZFA,CTR,M
...,...,...,...
1202,PDZA968YLU,PD,M
1203,PDBE349TEN,CTR,M
1204,PDCJ314NKT,CTR,F
1205,PDYZ819YLQ,PD,M


In [27]:
index_pdbp.reset_index(inplace=True)

In [36]:
index_pdbp.rename(columns={"index":"ID"},inplace=True)

In [41]:
index_pdbp.set_index('ID', inplace=True)

In [42]:
index_pdbp

,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000285687,ENSG00000285765,ENSG00000285814,ENSG00000285843,ENSG00000285891,ENSG00000285913,ENSG00000285937,ENSG00000285946,ENSG00000285950,ENSG00000285975
ID,,,,,,,,,,,,,,,,,,,,,
PDLK184WGE,-0.795371,-2.729347,4.132727,5.947212,3.915167,9.081336,2.544537,4.212770,5.115425,6.266301,...,-2.784321,-2.838762,-2.786100,-2.942664,-2.770755,-2.770755,-2.788866,-2.887931,-2.825500,-2.781080
PDXW222GG2,0.029172,-3.487062,5.383310,6.113187,4.208315,8.994625,4.462823,4.380455,5.551275,5.987634,...,-3.433885,-3.367082,-3.431357,-3.268287,-3.452658,-3.452658,-3.427209,-3.317961,-3.380948,-3.442536
PDDY096RP8,0.161952,-2.245680,4.764646,6.189538,4.310707,8.915631,3.514179,4.579706,5.807383,6.550710,...,-3.481217,-3.462194,-3.480309,-3.362947,-3.490272,-3.490272,-3.479424,-3.199985,-3.393907,-3.483213
PDCJ149XWQ,-0.851656,-2.650620,5.398471,6.203822,4.209682,8.833031,3.559513,4.704305,5.508427,6.170992,...,-2.240673,-2.789659,-2.730225,-2.912417,-2.710112,-2.710112,-2.733033,-2.850490,-2.775262,-2.722415
PDYH176ZFA,-0.712838,-2.724630,4.980514,6.184612,4.256929,8.860026,2.689762,4.447423,5.452709,6.317944,...,-2.781719,-2.836982,-2.783305,-2.940982,-2.767595,-2.767595,-2.785704,-2.886320,-2.823342,-2.777919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PDZA968YLU,-0.432641,-2.489323,4.779148,5.963757,4.270477,9.142878,2.490018,4.810756,5.216479,6.212650,...,-2.589854,-2.695303,-2.593489,-2.857318,-2.565912,-2.565912,-2.597467,-2.775536,-2.670709,-2.583686
PDBE349TEN,-0.769956,-2.580791,4.700159,5.839763,3.898746,9.267990,4.262164,4.266996,5.253896,6.150312,...,-2.670956,-2.754169,-2.674132,-2.891556,-2.649150,-2.649150,-2.678160,-2.821396,-2.735409,-2.664074
PDCJ314NKT,-0.075972,-3.483480,4.797046,6.283134,4.454279,8.577536,3.152542,4.022217,5.745774,6.317370,...,-3.419326,-3.350476,-3.416596,-3.247988,-3.442413,-3.442413,-3.414100,-3.301836,-3.365560,-3.424378


In [44]:
combined_pdbp = pd.merge(index_pdbp, pdbp_ppmi_data3, on="ID")

In [75]:
combined_pdbp.to_csv(path1/"pdbp_data.csv",sep='\t')

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_selection import SelectFromModel

# Load your dataset
data = pd.read_csv('your_dataset.csv')
X = data.drop('target', axis=1)  # Replace 'target' with your target column name
y = data['target']

# Initialize CatBoost model
model = CatBoostClassifier(iterations=1000, 
                           learning_rate=0.1, 
                           depth=6, 
                           loss_function='Logloss', 
                           verbose=0)

# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Fit model
model.fit(X, y)

# Get feature importances
feature_importances = model.get_feature_importance(Pool(X, label=y))

# Select features
selector = SelectFromModel(model, prefit=True, threshold='mean')
X_new = selector.transform(X)

# Get selected feature names
selected_features = X.columns[selector.get_support()]
print("Selected features:", selected_features)

# Create a new CatBoost model
model_selected = CatBoostClassifier(iterations=1000, 
                                    learning_rate=0.1, 
                                    depth=6, 
                                    loss_function='Logloss', 
                                    verbose=0)

# Cross-validation scores with selected features
scores = cross_val_score(model_selected, X_new, y, cv=cv, scoring='accuracy')
print("Cross-validation accuracy with selected features:", scores.mean())


FileNotFoundError: [Errno 2] No such file or directory: 'your_dataset.csv'

In [1]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

# Load dataset
X, y = load_iris(return_X_y=True)

# Initialize model
model = RandomForestClassifier()

# Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
skf_scores = cross_val_score(model, X, y, cv=skf, scoring='accuracy')
print(f'Stratified K-Fold CV scores: {skf_scores}')
print(f'Mean accuracy: {np.mean(skf_scores):.4f}, Standard deviation: {np.std(skf_scores):.4f}')

# Repeated Stratified K-Fold Cross Validation
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
rskf_scores = cross_val_score(model, X, y, cv=rskf, scoring='accuracy')
print(f'Repeated Stratified K-Fold CV scores: {rskf_scores}')
print(f'Mean accuracy: {np.mean(rskf_scores):.4f}, Standard deviation: {np.std(rskf_scores):.4f}')


Stratified K-Fold CV scores: [1.         0.96666667 0.93333333 0.96666667 0.9       ]
Mean accuracy: 0.9533, Standard deviation: 0.0340
Repeated Stratified K-Fold CV scores: [1.         0.96666667 0.93333333 0.96666667 0.9        0.93333333
 0.96666667 0.9        0.96666667 0.96666667 0.9        0.96666667
 1.         0.96666667 0.96666667 0.9        0.93333333 1.
 0.93333333 0.93333333 0.93333333 0.93333333 0.96666667 1.
 0.93333333 0.96666667 0.93333333 0.9        0.9        1.
 0.93333333 0.96666667 0.96666667 0.96666667 0.9        0.96666667
 0.96666667 0.93333333 0.96666667 0.9        0.96666667 0.93333333
 0.9        0.96666667 0.93333333 0.9        0.96666667 0.93333333
 1.         0.93333333]
Mean accuracy: 0.9473, Standard deviation: 0.0313


In [1]:
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score
